# Neural machine translation

In [1]:
import tensorflow as tf

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from sklearn.model_selection import train_test_split

import unicodedata
import re
import numpy as np
import os
import io
import time

## Download and prepare the dataset

We'll use a language dataset provided by http://www.manythings.org/anki/

In [2]:
!wget http://www.manythings.org/anki/rus-eng.zip

--2023-02-16 16:48:55--  http://www.manythings.org/anki/rus-eng.zip
Resolving www.manythings.org (www.manythings.org)... 173.254.30.110
Connecting to www.manythings.org (www.manythings.org)|173.254.30.110|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 15011848 (14M) [application/zip]
Saving to: 'rus-eng.zip'

     0K .......... .......... .......... .......... ..........  0% 5,26K 46m15s
    50K .......... .......... .......... .......... ..........  0% 8,97K 36m34s
   100K .......... .......... .......... .......... ..........  1% 12,4K 30m49s
   150K .......... .......... .......... .......... ..........  1% 12,3K 27m57s
   200K .......... .......... .......... .......... ..........  1% 12,7K 26m3s
   250K .......... .......... .......... .......... ..........  2% 27,1K 23m6s
   300K .......... .......... .......... .......... ..........  2% 15,6K 21m56s
   350K .......... .......... .......... .......... ..........  2% 19,4K 20m39s
   400K .......... ......

In [3]:
!mkdir rus-eng
!unzip rus-eng.zip -d rus-eng/

"unzip" ­Ґ пў«пҐвбп ў­гваҐ­­Ґ© Ё«Ё ў­Ґи­Ґ©
Є®¬ ­¤®©, ЁбЇ®«­пҐ¬®© Їа®Ја ¬¬®© Ё«Ё Ї ЄҐв­л¬ д ©«®¬.


In [4]:
!ls /content/rus-eng/ -lah

"ls" ­Ґ пў«пҐвбп ў­гваҐ­­Ґ© Ё«Ё ў­Ґи­Ґ©
Є®¬ ­¤®©, ЁбЇ®«­пҐ¬®© Їа®Ја ¬¬®© Ё«Ё Ї ЄҐв­л¬ д ©«®¬.


In [12]:
# Download the file
path_to_file = "rus-eng/rus.txt"

In [13]:
def preprocess_sentence(w):
  w = w.lower().strip()

  # creating a space between a word and the punctuation following it
  # eg: "he is a boy." => "he is a boy ."
  # Reference:- https://stackoverflow.com/questions/3645931/python-padding-punctuation-with-white-spaces-keeping-punctuation
  w = re.sub(r"([?.!,])", r" \1 ", w)
  w = re.sub(r'[" "]+', " ", w)

  # replacing everything with space except (a-z, A-Z, ".", "?", "!", ",")
  w = re.sub(r"[^a-zA-Zа-яА-Я?.!,']+", " ", w)

  w = w.strip()

  # adding a start and an end token to the sentence
  # so that the model know when to start and stop predicting.
  w = '<start> ' + w + ' <end>'
  return w

In [14]:
preprocess_sentence("I can't go.")

"<start> i can't go . <end>"

In [15]:
# 1. Remove the accents
# 2. Clean the sentences
# 3. Return word pairs in the format: [ENG, RUS]
def create_dataset(path, num_examples):
  lines = io.open(path, encoding='UTF-8').read().strip().split('\n')

  word_pairs = [[preprocess_sentence(w) for w in l.split('\t')[:2]]  for l in lines[:num_examples]]

  return zip(*word_pairs)

In [16]:
en, ru = create_dataset(path_to_file, None)
print(en[0])
print(ru[0])

<start> go . <end>
<start> марш ! <end>


In [17]:
def tokenize(lang):
  lang_tokenizer = tf.keras.preprocessing.text.Tokenizer(
      filters='')
  lang_tokenizer.fit_on_texts(lang)

  tensor = lang_tokenizer.texts_to_sequences(lang)

  tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor,
                                                         padding='post')

  return tensor, lang_tokenizer

In [18]:
def load_dataset(path, num_examples=None):
  # creating cleaned input, output pairs
  targ_lang, inp_lang = create_dataset(path, num_examples)

  input_tensor, inp_lang_tokenizer = tokenize(inp_lang)
  target_tensor, targ_lang_tokenizer = tokenize(targ_lang)

  return input_tensor, target_tensor, inp_lang_tokenizer, targ_lang_tokenizer

### Limit the size of the dataset to experiment faster (optional)


In [19]:
len(en), len(ru)

(451436, 451436)

In [20]:
# Try experimenting with the size of that dataset
num_examples = 100000
input_tensor, target_tensor, inp_lang, targ_lang = load_dataset(path_to_file, num_examples)

# Calculate max_length of the target tensors
max_length_targ, max_length_inp = target_tensor.shape[1], input_tensor.shape[1]

In [21]:
# Creating training and validation sets using an 80-20 split
input_tensor_train, input_tensor_val, target_tensor_train, target_tensor_val = train_test_split(input_tensor, target_tensor, test_size=0.2)

# Show length
print(len(input_tensor_train), len(target_tensor_train), len(input_tensor_val), len(target_tensor_val))

80000 80000 20000 20000


In [22]:
def convert(lang, tensor):
  for t in tensor:
    if t!=0:
      print ("%d ----> %s" % (t, lang.index_word[t]))

In [23]:
print ("Input Language; index to word mapping")
convert(inp_lang, input_tensor_train[0])
print ()
print ("Target Language; index to word mapping")
convert(targ_lang, target_tensor_train[0])

Input Language; index to word mapping
1 ----> <start>
8 ----> это
15366 ----> леопард
5 ----> ?
2 ----> <end>

Target Language; index to word mapping
1 ----> <start>
8 ----> is
18 ----> that
9 ----> a
5956 ----> leopard
6 ----> ?
2 ----> <end>


### Create a tf.data dataset

In [24]:
BUFFER_SIZE = len(input_tensor_train)
BATCH_SIZE = 256
steps_per_epoch = len(input_tensor_train)//BATCH_SIZE
embedding_dim = 500
units = 2024
vocab_inp_size = len(inp_lang.word_index)+1
vocab_tar_size = len(targ_lang.word_index)+1

dataset = tf.data.Dataset.from_tensor_slices((input_tensor_train, target_tensor_train)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)

In [25]:
example_input_batch, example_target_batch = next(iter(dataset))
example_input_batch.shape, example_target_batch.shape

(TensorShape([256, 15]), TensorShape([256, 11]))

In [26]:
class Encoder(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, enc_units, batch_sz):
    super(Encoder, self).__init__()
    self.batch_sz = batch_sz
    self.enc_units = enc_units
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(self.enc_units,
                                   return_sequences=False,
                                   return_state=True,
                                   recurrent_initializer='glorot_uniform')
    

  def call(self, x, hidden):
    x = self.embedding(x)
    output, state = self.gru(x, initial_state = hidden)
    return state

  def initialize_hidden_state(self):
    return tf.zeros((self.batch_sz, self.enc_units))

In [27]:
encoder = Encoder(vocab_inp_size, embedding_dim, units, BATCH_SIZE)

# sample input
sample_hidden = encoder.initialize_hidden_state()
sample_hidden = encoder(example_input_batch, sample_hidden)
# print ('Encoder output shape: (batch size, sequence length, units) {}'.format(sample_output.shape))
print ('Encoder Hidden state shape: (batch size, units) {}'.format(sample_hidden.shape))

Encoder Hidden state shape: (batch size, units) (256, 2024)


In [28]:
class Decoder(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, dec_units, batch_sz):
    super(Decoder, self).__init__()
    self.batch_sz = batch_sz
    self.dec_units = dec_units
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(self.dec_units,
                                   return_sequences=True,
                                   return_state=True,
                                   recurrent_initializer='glorot_uniform')
    self.fc = tf.keras.layers.Dense(vocab_size)

  def call(self, x, hidden):
    # enc_output shape == (batch_size, max_length, hidden_size)

    # x shape after passing through embedding == (batch_size, 1, embedding_dim)
    x = self.embedding(x)

    # x shape after concatenation == (batch_size, 1, embedding_dim + hidden_size)

    # passing the concatenated vector to the GRU
    output, state = self.gru(x, initial_state=hidden)

    # output shape == (batch_size * 1, hidden_size)
    output = tf.reshape(output, (-1, output.shape[2]))

    # output shape == (batch_size, vocab)
    x = self.fc(output)

    return x, state

In [29]:
decoder = Decoder(vocab_tar_size, embedding_dim, units, BATCH_SIZE)

decoder_sample_x, decoder_sample_h = decoder(tf.random.uniform((BATCH_SIZE, 1)),
                                      sample_hidden)



In [30]:
decoder_sample_x.shape

TensorShape([256, 7335])

In [31]:
decoder_sample_h.shape

TensorShape([256, 2024])

## Define the optimizer and the loss function

In [32]:
optimizer = tf.keras.optimizers.Adam()

loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

def loss_function(real, pred):
  mask = tf.math.logical_not(tf.math.equal(real, 0))
  loss_ = loss_object(real, pred)

  mask = tf.cast(mask, dtype=loss_.dtype)
  loss_ *= mask

  return tf.reduce_mean(loss_)

## Checkpoints (Object-based saving)

In [33]:
checkpoint_dir = './training_nmt_checkpoints'

checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")

checkpoint = tf.train.Checkpoint(optimizer=optimizer,
                                 encoder=encoder,
                                 decoder=decoder)

In [34]:
@tf.function
def train_step(inp, targ, enc_hidden):
  loss = 0

  with tf.GradientTape() as tape:
    enc_hidden = encoder(inp, enc_hidden)

    dec_hidden = enc_hidden

    dec_input = tf.expand_dims([targ_lang.word_index['<start>']] * BATCH_SIZE, 1)

    # Teacher forcing - feeding the target as the next input
    for t in range(1, targ.shape[1]):
      # passing enc_output to the decoder
      predictions, dec_hidden = decoder(dec_input, dec_hidden)

      loss += loss_function(targ[:, t], predictions)

      # using teacher forcing
      dec_input = tf.expand_dims(targ[:, t], 1)

  batch_loss = (loss / int(targ.shape[1]))

  variables = encoder.trainable_variables + decoder.trainable_variables

  gradients = tape.gradient(loss, variables)

  optimizer.apply_gradients(zip(gradients, variables))

  return batch_loss

In [35]:
EPOCHS = 50

for epoch in range(EPOCHS):
  start = time.time()

  enc_hidden = encoder.initialize_hidden_state()
  total_loss = 0

  for (batch, (inp, targ)) in enumerate(dataset.take(steps_per_epoch)):
    batch_loss = train_step(inp, targ, enc_hidden)
    total_loss += batch_loss

    if batch % 100 == 0:
      print('Epoch {} Batch {} Loss {:.4f}'.format(epoch + 1,
                                                   batch,
                                                   batch_loss.numpy()))
  # saving (checkpoint) the model every 2 epochs
  if (epoch + 1) % 2 == 0:
    checkpoint.save(file_prefix = checkpoint_prefix)

  print('Epoch {} Loss {:.4f}'.format(epoch + 1,
                                      total_loss / steps_per_epoch))
  print('Time taken for 1 epoch {} sec\n'.format(time.time() - start))

Epoch 1 Batch 0 Loss 4.7288
Epoch 1 Batch 100 Loss 1.7682
Epoch 1 Batch 200 Loss 1.4790
Epoch 1 Batch 300 Loss 1.1995
Epoch 1 Loss 1.6698
Time taken for 1 epoch 49.354281425476074 sec

Epoch 2 Batch 0 Loss 1.1668
Epoch 2 Batch 100 Loss 1.0227
Epoch 2 Batch 200 Loss 0.8969
Epoch 2 Batch 300 Loss 0.8673
Epoch 2 Loss 0.9793
Time taken for 1 epoch 38.56188368797302 sec

Epoch 3 Batch 0 Loss 0.7247
Epoch 3 Batch 100 Loss 0.6896
Epoch 3 Batch 200 Loss 0.6649
Epoch 3 Batch 300 Loss 0.5736
Epoch 3 Loss 0.6557
Time taken for 1 epoch 36.70270562171936 sec

Epoch 4 Batch 0 Loss 0.4275
Epoch 4 Batch 100 Loss 0.4540
Epoch 4 Batch 200 Loss 0.3975
Epoch 4 Batch 300 Loss 0.3906
Epoch 4 Loss 0.4218
Time taken for 1 epoch 38.392359018325806 sec

Epoch 5 Batch 0 Loss 0.2830
Epoch 5 Batch 100 Loss 0.2866
Epoch 5 Batch 200 Loss 0.3072
Epoch 5 Batch 300 Loss 0.3035
Epoch 5 Loss 0.2712
Time taken for 1 epoch 36.783775806427 sec

Epoch 6 Batch 0 Loss 0.1801
Epoch 6 Batch 100 Loss 0.1786
Epoch 6 Batch 200 Loss

Epoch 44 Loss 0.0412
Time taken for 1 epoch 40.53286671638489 sec

Epoch 45 Batch 0 Loss 0.0318
Epoch 45 Batch 100 Loss 0.0378
Epoch 45 Batch 200 Loss 0.0445
Epoch 45 Batch 300 Loss 0.0480
Epoch 45 Loss 0.0407
Time taken for 1 epoch 37.44020652770996 sec

Epoch 46 Batch 0 Loss 0.0274
Epoch 46 Batch 100 Loss 0.0313
Epoch 46 Batch 200 Loss 0.0649
Epoch 46 Batch 300 Loss 0.0389
Epoch 46 Loss 0.0406
Time taken for 1 epoch 40.435800313949585 sec

Epoch 47 Batch 0 Loss 0.0324
Epoch 47 Batch 100 Loss 0.0321
Epoch 47 Batch 200 Loss 0.0445
Epoch 47 Batch 300 Loss 0.0389
Epoch 47 Loss 0.0409
Time taken for 1 epoch 37.81082892417908 sec

Epoch 48 Batch 0 Loss 0.0305
Epoch 48 Batch 100 Loss 0.0383
Epoch 48 Batch 200 Loss 0.0427
Epoch 48 Batch 300 Loss 0.0406
Epoch 48 Loss 0.0409
Time taken for 1 epoch 40.21131730079651 sec

Epoch 49 Batch 0 Loss 0.0255
Epoch 49 Batch 100 Loss 0.0408
Epoch 49 Batch 200 Loss 0.0386
Epoch 49 Batch 300 Loss 0.0392
Epoch 49 Loss 0.0396
Time taken for 1 epoch 37.4962356

## Translate

* The evaluate function is similar to the training loop, except we don't use *teacher forcing* here. The input to the decoder at each time step is its previous predictions along with the hidden state and the encoder output.
* Stop predicting when the model predicts the *end token*.
* And store the *attention weights for every time step*.

Note: The encoder output is calculated only once for one input.

In [36]:
def evaluate(sentence):
  attention_plot = np.zeros((max_length_targ, max_length_inp))

  sentence = preprocess_sentence(sentence)

  inputs = [inp_lang.word_index[i] for i in sentence.split(' ')]
  inputs = tf.keras.preprocessing.sequence.pad_sequences([inputs],
                                                         maxlen=max_length_inp,
                                                         padding='post')
  inputs = tf.convert_to_tensor(inputs)

  result = ''

  hidden = [tf.zeros((1, units))]
  enc_hidden = encoder(inputs, hidden)

  dec_hidden = enc_hidden
  dec_input = tf.expand_dims([targ_lang.word_index['<start>']], 0)

  for t in range(max_length_targ):
    predictions, dec_hidden = decoder(dec_input, dec_hidden)

    # storing the attention weights to plot later on
    predicted_id = tf.argmax(predictions[0]).numpy()
    result += targ_lang.index_word[predicted_id] + ' '

    if targ_lang.index_word[predicted_id] == '<end>':
      return result, sentence

    # the predicted ID is fed back into the model
    dec_input = tf.expand_dims([predicted_id], 0)

  return result, sentence

In [37]:
def translate(sentence):
  result, sentence = evaluate(sentence)

  print('Input: %s' % (sentence))
  print('Predicted translation: {}'.format(result))

## Restore the latest checkpoint and test

In [38]:
# restoring the latest checkpoint in checkpoint_dir
checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))

In [39]:
translate('Здесь хорошо.')

Input: <start> здесь хорошо . <end>
Predicted translation: it's good here . <end> 


In [40]:
translate('Я не смогу поехать.')

Input: <start> я не смогу поехать . <end>
Predicted translation: i can't go . <end> 


In [41]:
translate(u'Вы еще дома?')

Input: <start> вы еще дома ? <end>
Predicted translation: are you still at home ? <end> 


In [42]:
translate(u'Вы все еще дома?')

Input: <start> вы все еще дома ? <end>
Predicted translation: are you still home ? <end> 


In [43]:
translate(u'Попробуй сделать это.')

Input: <start> попробуй сделать это . <end>
Predicted translation: try to do that . <end> 


In [44]:
translate(u'Я люблю, когда идет снег.')

Input: <start> я люблю , когда идет снег . <end>
Predicted translation: i like her badly . <end> 


In [46]:
translate(u'Я люблю, когда падает снег.')

Input: <start> я люблю , когда падает снег . <end>
Predicted translation: i like her badly lost . <end> 


In [45]:
translate(u'Я никогда такого не делаю.')

Input: <start> я никогда такого не делаю . <end>
Predicted translation: i never do that . <end> 


In [50]:
translate(u'Я хочу поесть мясо.')

Input: <start> я хочу поесть мясо . <end>
Predicted translation: i want to eat meat . <end> 
